In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('GBR_SCO.Mx_1x1.txt', sep='\s+', skiprows=2)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/ck/fnvjp8v922ddrgqn5bbj6tvh0000gn/T/ipykernel_26533/2534155762.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('GBR_SCO.Mx_1x1.txt', sep='\s+', skiprows=2)


In [19]:
# Convert the 'Age' column to numeric, forcing errors to NaN
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Drop rows with NaN values in 'Age' column
df = df.dropna(subset=['Age'])

# Pivot the dataframe to create a matrix q_x,t with Age as rows and Year as columns
q_matrix = df.pivot(index='Age', columns='Year', values='Total')

# Convert the values to numeric, forcing errors to NaN
q_matrix = q_matrix.apply(pd.to_numeric, errors='coerce')

# Sort the ages in ascending order
q_matrix = q_matrix.sort_index()

# Keep only the years from 1950 to 2000
q_matrix = q_matrix.loc[0:99, 1950:2000]

# Display the resulting matrix
q_matrix

Year,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
Age,,,,,,,,,,,,,,,,,,,,,
0.0,0.039137,0.038249,0.036678,0.032296,0.032244,0.031758,0.029922,0.030325,0.029193,0.029118,...,0.007218,0.006816,0.006406,0.006093,0.006208,0.006204,0.005334,0.005549,0.004921,0.005660
1.0,0.003033,0.003083,0.002502,0.002471,0.002027,0.002049,0.001894,0.001695,0.001475,0.002073,...,0.000830,0.000629,0.000457,0.000515,0.000385,0.000399,0.000324,0.000390,0.000384,0.000323
2.0,0.001697,0.001791,0.001393,0.001358,0.001455,0.001031,0.000973,0.000930,0.000925,0.001055,...,0.000343,0.000470,0.000399,0.000198,0.000235,0.000306,0.000217,0.000291,0.000204,0.000280
3.0,0.001539,0.001281,0.001080,0.000995,0.000776,0.000873,0.000778,0.000933,0.000785,0.000930,...,0.000335,0.000296,0.000282,0.000261,0.000229,0.000173,0.000194,0.000235,0.000240,0.000085
4.0,0.001117,0.001007,0.000999,0.000888,0.000795,0.000803,0.000641,0.000580,0.000602,0.000685,...,0.000244,0.000168,0.000327,0.000125,0.000138,0.000169,0.000173,0.000146,0.000151,0.000241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.0,0.434222,0.466286,0.392405,0.372893,0.449926,0.428451,0.468887,0.415256,0.415707,0.426915,...,0.298511,0.296732,0.326174,0.300096,0.294802,0.276661,0.286471,0.318880,0.320380,0.312072
96.0,0.452496,0.552954,0.415271,0.414771,0.493203,0.510970,0.515742,0.441927,0.458163,0.462943,...,0.319735,0.340761,0.370594,0.345366,0.333772,0.334249,0.310435,0.318505,0.349515,0.330229
97.0,0.402243,0.641228,0.436268,0.482568,0.510317,0.606176,0.545504,0.431213,0.495993,0.490133,...,0.306937,0.374410,0.420982,0.371763,0.359360,0.352035,0.366098,0.364604,0.386620,0.324557


In [16]:


# Define the original matrix M
M = q_matrix.to_numpy()

# Compute the mean of M along the rows and keep the dimensions
a_fit = np.mean(M, axis=1, keepdims=True)

# Perform Singular Value Decomposition (SVD) on the mean-centered matrix
u, s, vt = np.linalg.svd(M - a_fit, full_matrices=False)

# Extract the first left singular vector u1
u1 = u[:, 0]

# Extract the first right singular vector v1
v1 = vt[0, :]

# Extract the first singular value s1
s1 = s[0]

# Define b_fit as the first left singular vector (same as u1)
b_fit = u1

# Define k_fit as the product of the first singular value and the first right singular vector
k_fit = s1 * v1

# Reconstruct the matrix pred using the outer product of b_fit and k_fit, added to the mean a_fit
pred = a_fit + np.outer(b_fit, k_fit)


In [18]:
pd.DataFrame(pred)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.021792,0.044843,0.022749,0.030659,0.034200,0.039653,0.030859,0.020445,0.026474,0.027482,...,0.007252,0.009944,0.012915,0.008213,0.009693,0.009550,0.010232,0.008653,0.011786,0.007458
1,0.001477,0.003028,0.001542,0.002074,0.002312,0.002679,0.002087,0.001387,0.001792,0.001860,...,0.000499,0.000680,0.000880,0.000564,0.000664,0.000654,0.000700,0.000594,0.000804,0.000513
2,0.000860,0.001742,0.000897,0.001199,0.001335,0.001544,0.001207,0.000809,0.001039,0.001078,...,0.000304,0.000407,0.000520,0.000340,0.000397,0.000392,0.000418,0.000357,0.000477,0.000312
3,0.000696,0.001341,0.000723,0.000944,0.001043,0.001196,0.000950,0.000659,0.000827,0.000855,...,0.000290,0.000365,0.000448,0.000317,0.000358,0.000354,0.000373,0.000329,0.000417,0.000295
4,0.000567,0.001109,0.000589,0.000775,0.000859,0.000987,0.000780,0.000535,0.000677,0.000701,...,0.000225,0.000288,0.000358,0.000248,0.000282,0.000279,0.000295,0.000258,0.000332,0.000230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.378275,0.492568,0.383021,0.422238,0.439797,0.466835,0.423231,0.371595,0.401491,0.406489,...,0.306181,0.319532,0.334261,0.310948,0.318287,0.317575,0.320956,0.313127,0.328665,0.307204
96,0.417863,0.570163,0.424188,0.476446,0.499844,0.535874,0.477769,0.408962,0.448800,0.455460,...,0.321795,0.339585,0.359213,0.328147,0.337927,0.336979,0.341484,0.331051,0.351756,0.323159
97,0.435819,0.614347,0.443233,0.504491,0.531919,0.574152,0.506041,0.425385,0.472083,0.479891,...,0.323207,0.344061,0.367068,0.330653,0.342116,0.341005,0.346286,0.334057,0.358327,0.324805
98,0.462976,0.643919,0.470490,0.532577,0.560376,0.603181,0.534148,0.452401,0.499731,0.507644,...,0.348841,0.369976,0.393295,0.356387,0.368006,0.366879,0.372232,0.359837,0.384436,0.350460
